In [1]:
from PyPDF2 import PdfReader
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
splitter = RecursiveCharacterTextSplitter(
            separators=["\\n\\n", "\\n", " ", ""],
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len)
pdf_reader = PdfReader("Reports/tcs.pdf")


In [24]:
metadatas=[]
text_by_loaders_page=[]

In [25]:
for i, page in enumerate(pdf_reader.pages):
            text = page.extract_text()
            text_temp = []
            for page_text in text.split('\x0c'):
                text_temp += splitter.split_text(page_text)
            for j, chunk in enumerate(text_temp):
                metadatas.append({'page': i+1, 'index': j+1})
            pages = text_by_loaders_page.append(text_temp)

In [28]:
text_by_loaders_page

[['Innovate, Adapt, Thrive\nIntegrated Annual Report\n2022-23'],
 ['About TCS\nTata Consultancy Services is an IT services, consulting and \nbusiness solutions organization that has been partnering with many of the world’s largest businesses in their transformation journeys for over 55 years. Its consulting-led, cognitive powered, portfolio of business, technology and engineering services and solutions is delivered through its unique Location Independent Agile™ delivery model, recognized as a benchmark of excellence in software development.\nA part of the Tata group, India’s largest multinational \nbusiness group, TCS has over 614,000 of the world’s best-trained consultants in 55 countries. The company generated consolidated revenues of US $27.9 billion in the fiscal year ended March 31, 2023, and is listed on the BSE and the NSE in India. \nTCS’ proactive stance on climate change and award-winning \nwork with communities across the world have earned it a place in leading sustainabilit

In [2]:
import fitz  # import package PyMuPDF

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = fitz.open("Reports/tcs.pdf")

In [3]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
text = ""
for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
text=re.sub(r'\n', ' ', text)

In [64]:
# text_splitter = SemanticChunker(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))
# chunks = text_splitter.split_text(text=text)

In [20]:
text_Rec_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            chunk_overlap=200,
            length_function=len
            )

In [16]:
Recchunks = text_Rec_splitter.split_text(text=text)

In [17]:
len(Recchunks)

597

In [19]:
Recchunks

['Innovate, Adapt, Thrive Integrated Annual Report 2022-23 About TCS Tata Consultancy Services is an IT services, consulting and  business solutions organization that has been partnering with  many of the world’s largest businesses in their transformation  journeys for over 55 years. Its consulting-led, cognitive powered,  portfolio of business, technology and engineering services and  solutions is delivered through its unique Location Independent  Agile™ delivery model, recognized as a benchmark of excellence  in software development. A part of the Tata group, India’s largest multinational  business group, TCS has over 614,000 of the world’s best- trained consultants in 55 countries. The company generated  consolidated revenues of US $27.9 billion in the fiscal year  ended March 31, 2023, and is listed on the BSE and the NSE in  India.  TCS’ proactive stance on climate change and award-winning  work with communities across the world have earned it a  place in leading sustainability in

In [72]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [73]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [74]:
VectorStore = FAISS.from_texts(Recchunks, embedding=embeddings)

In [75]:
doc = VectorStore.similarity_search("Who is CEO of TCS")

In [76]:
doc

[Document(page_content='Unit to boost its  modernization plans. TCS will manage and ramp down over 70%  of Digital’s legacy technology estate and boost its capacity to  accelerate the build of its new strategic technology architecture,  supporting the Group’s growth. Won the 2022 Microsoft Supplier of the Year award in the large  supplier category. TCS was recognized for consistently delivering  new capabilities with automation solutions that help improve  accuracy for Microsoft, while also increasing its speed to market.  TCS was cited for its expertise across industries that enables  Microsoft to better serve customers, showcasing what it means  to focus on strategy, impact, agility, and modernization. Tata Group Chairperson and TCS Chairman, N Chandrasekaran,  was bestowed with the Eisenhower Global Leadership Award  for his contribution to commerce, by the non-profit, Business  Council for International Understanding. Hans Vestberg, the  Chairman and CEO of Verizon, presented N Cha

In [1]:
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import pickle

In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GradientLLM

In [3]:
import os
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    # Access token under https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("gradient.ai access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    # `ID` listed in `$ gradient workspace list`
    # also displayed after login at at https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("gradient.ai workspace id:")

In [ ]:
# gradient accesst token qmo0P8rEERRVzm0XoEIvm9j41aQEu5Z2

In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [5]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model="6fd9d674-c159-418f-98d7-689af2b11715_model_adapter",
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    model_kwargs=dict(max_generated_token_count=400,temperature= 0.75, max_length= 3000, Stream=True),
    # callback_manager= callback_manager,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [7]:
from langchain_core.prompts import ChatPromptTemplate
template = """You are a financial expert with access to the annual report of the company.
When answering questions about the company's financial performance, prioritize information from the Financial Statements section.Considering the user's question, provide clear and concise answers from given context.
{context}

Question: {question}
Answer:
"""
# template = """You are a financial expert with access to the annual report of the company.
# When answering questions about the company's financial performance, prioritize information from the Financial Statements section.

# {context}

# **Question:** {question}

# **Answer:**

# If you can find relevant information in the annual report, please provide a clear and concise answer based on the facts.
# If you're unable to find an answer in the report, you can respond by saying:

# * "I couldn't find information related to '{question}' in the annual report."
# * "The annual report doesn't provide sufficient details to answer this question definitively."

# """


prompt = ChatPromptTemplate.from_template(template)

In [8]:
import pickle

In [10]:
with open(f"vectorestore/bSnasyo2HywpAgXnz29e.pkl", "rb") as f:
        VectorStore = pickle.load(f)

In [11]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm, VectorStore.as_retriever(), return_source_documents=True,combine_docs_chain_kwargs={"prompt": prompt})

In [12]:
chat_history = []

In [18]:
query = "What is mention in letter of ceo"
result=chain.invoke({"question": query, "chat_history": chat_history})

In [19]:
text=result['answer']

In [20]:
print(text)

 The letter from the CEO highlights the company's achievements in the previous year, including:

1. Revenue growth of 13.5% in constant currency
2. Operating margin of 25.5%
3. Net margin of 18.2%
4. Robust demand environment with 16% sequential growth in Q4
5. Strong focus on innovation, with 280 patents filed in FY23
6. Continued investment in talent, with 3.3x employee growth in the past 5 years
7. Increased market share in key industries like Financial Services, Communications, Media, and Technology (F500), and Commercial Banking
8. Strong commitment to sustainability, with a 56% reduction in greenhouse gas emissions since 2015 
9. Launch of the TCS vision 2025, a strategy to deliver 25 billion dollars in revenue by 2025, with a focus on growth, operational efficiency, and investment in emerging technologies such as AI, ML, Analytics, Cloud, and Edge computing.

The CEO also highlights the company's focus on talent acquisition and development, as well as its commitment to sustainab

In [119]:
print(text)

 Based on the context provided, I can answer the following questions with certainty:

* Who is the CEO of TCS?
Answer: The CEO of TCS is K Krithivasan, effective from June 1, 2023.

However, for the following questions, I must admit that I do not have the answer:

* Who is the Chairman of TCS?
* Who is the Chairperson of TCS?
* Who is the Group Chairperson of TCS?
* Who is the Eisenhower Global Leadership Award winner?
* Who is the Chairman and CEO of Verizon?

Please provide me with more context or information on these questions, and I will do my best to provide a helpful answer.
